In [1]:
!pip install -qqq transformers datasets wandb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 61.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 54.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 97.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 40.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 121.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 57.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 18.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 20.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.6/190.6 kB 25.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 241.0/241.0 kB 31.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 37.

In [2]:
!pip install transformers[torch] --upgrade

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.1/258.1 kB 5.8 MB/s eta 0:00:00


In [3]:
!pip install accelerate -U

In [4]:
!pip install -qqq unidecode contractions pyspellchecker wordninja symspellpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.5/235.5 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 14.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 541.6/541.6 kB 19.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 24.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.2/57.2 kB 6.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.9/289.9 kB 33.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.8/110.8 kB 14.7 MB/s eta 0:00:00


#imports

In [5]:
import pandas as pd
import numpy as np
import contractions as contract
import spacy
import re
import torch
import torch.nn as nn
import wandb

#utils
import transformers
from transformers import AutoModelForSequenceClassification, AutoTokenizer
from transformers import TrainingArguments, Trainer
from datasets import Dataset, DatasetDict, load_metric
from sklearn.model_selection import train_test_split
import pkg_resources
from spellchecker import SpellChecker
from symspellpy import SymSpell, Verbosity
import nltk
import unidecode
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
import accelerate
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Task/data/Suicide_Detection.csv', index_col=0)
df.reset_index(drop=True, inplace=True)
df.head()

,text,class
0,Ex Wife Threatening SuicideRecently I left my ...,suicide
1,Am I weird I don't get affected by compliments...,non-suicide
2,Finally 2020 is almost over... So I can never ...,non-suicide
3,i need helpjust help me im crying so hard,suicide
4,"I’m so lostHello, my name is Adam (16) and I’v...",suicide


In [ ]:
nlp = spacy.load("en_core_web_sm")
sym_spell = SymSpell(max_dictionary_edit_distance=2, prefix_length=7)

dictionary_path = pkg_resources.resource_filename("symspellpy", "frequency_dictionary_en_82_765.txt")
bigram_path = pkg_resources.resource_filename("symspellpy", "frequency_bigramdictionary_en_243_342.txt")

sym_spell.load_dictionary(dictionary_path, term_index=0, count_index=1)
sym_spell.load_bigram_dictionary(bigram_path, term_index=0, count_index=2)

def clean_text(text: str, lemmatize: bool = True) -> str:
    text = text.lower()
    text = text.strip()
    url_pattern = r'https?://\S+|www\.\S+'
    text = re.sub(url_pattern, '', text)
    text = unidecode.unidecode(text)
    text = contract.fix(text)
    suggestions = sym_spell.lookup_compound(text, max_edit_distance=2)
    correctedtext = suggestions[0].term
    text = correctedtext

    words = [token.text for token in nlp(text)]

    words = [word for word in words if word.isalnum()]
    words = [word for word in words if not word.isdigit()]

    stop_words = set(stopwords.words('english'))
    excluded_words = ["so", "no", "not", "but", "however", "although"]
    greeting_words = ["hello", "hi", "hey", "greetings", "good", "morning", "afternoon", "evening"]

    for word in excluded_words:
        stop_words.discard(word)
    words = [word for word in words if word not in stop_words]
    words = [word for word in words if word.lower() not in greeting_words]

    if lemmatize:
        lemmatizer = WordNetLemmatizer()
        words = [lemmatizer.lemmatize(word) for word in words]

    cleaned_text = ' '.join(words)
    return cleaned_text

In [ ]:
df['cleaned_text'] = df['text'].apply(lambda x: clean_text(x))
df.head(10)

In [ ]:
df.to_csv('/content/drive/MyDrive/Text_pred/Data/My_New_Cleaned_Suicide_Detection.csv', index=False)

In [85]:
df = pd.read_csv('/content/drive/MyDrive/Text_pred/Data/My_New_Cleaned_Suicide_Detection.csv', index_col=0)
df.reset_index(drop=True, inplace=True)
df.head()

,class,cleaned_text
0,suicide,sex wife threatening suicide recently left wif...
1,non-suicide,weird not get affected compliment coming someo...
2,non-suicide,finally almost so never hear bad year ever swe...
3,suicide,need help help cry so hard
4,suicide,so lost name adam struggling year afraid past ...


In [87]:
df.shape

(232074, 2)

In [88]:
df.isnull().sum()

class             0
cleaned_text    113
dtype: int64

In [89]:
df = df.dropna()

In [90]:
df.isnull().sum()

class           0
cleaned_text    0
dtype: int64

In [91]:
df['cleaned_text'] = df['cleaned_text'].astype(str)

<ipython-input-91-21e4be9e3b76>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['cleaned_text'] = df['cleaned_text'].astype(str)


In [92]:
posts_len = [len(x.split()) for x in df['cleaned_text']]
print(pd.Series(posts_len).describe())

count    231961.000000
mean         64.635844
std         116.365708
min           1.000000
25%          13.000000
50%          30.000000
75%          75.000000
max       19649.000000
dtype: float64


In [93]:
df.shape

(231961, 2)

In [94]:
df = df[df['cleaned_text'].apply(lambda x: len(x.split())<=100 and len(x.split())>=5)]
df.reset_index(drop=True, inplace=True)

In [95]:
df.shape

(183324, 2)

In [96]:
df.head()

,class,cleaned_text
0,suicide,sex wife threatening suicide recently left wif...
1,non-suicide,weird not get affected compliment coming someo...
2,non-suicide,finally almost so never hear bad year ever swe...
3,suicide,need help help cry so hard
4,suicide,end tonight not anymore quit


In [97]:
df.to_csv('/content/drive/MyDrive/Text_pred/Data/Final_Cleaned_Suicide_Detection.csv', index=True)

#Training

In [106]:
df = pd.read_csv('/content/drive/MyDrive/Text_pred/Data/Final_Cleaned_Suicide_Detection.csv', index_col=0)
df.reset_index(drop=True, inplace=True)
df.head()

,class,cleaned_text
0,suicide,sex wife threatening suicide recently left wif...
1,non-suicide,weird not get affected compliment coming someo...
2,non-suicide,finally almost so never hear bad year ever swe...
3,suicide,need help help cry so hard
4,suicide,end tonight not anymore quit


In [107]:
df.shape

(183324, 2)

In [108]:
device = torch.device("cuda")

In [109]:
EPOCHS = 1
BATCH_SIZE = 6
LEARNING_RATE = 1e-5
SEED = 4222

MODEL_SAVE_PATH = "/content/drive/MyDrive/Text_pred/Model/distilbert-base-uncased"
MODEL_CHECKPOINT_PATH = "/content/drive/MyDrive/Text_pred/Model/checkpoint"

WANDB_ENTITY = "nilankabiswas21"
WANDB_PROJECT = "suicide_detection"
WANDB_RUN = "distilbert-base-uncased"

In [110]:
df['label'] = df['class'].map({'suicide': 1, 'non-suicide': 0})
df

,class,cleaned_text,label
0,suicide,sex wife threatening suicide recently left wif...,1
1,non-suicide,weird not get affected compliment coming someo...,0
2,non-suicide,finally almost so never hear bad year ever swe...,0
3,suicide,need help help cry so hard,1
4,suicide,end tonight not anymore quit,1
...,...,...,...
183319,non-suicide,something today went sledding friend may not s...,0
183320,non-suicide,not like rock not going get anything but go,0
183321,non-suicide,tell so many friend not lonely everything depr...,0
183322,non-suicide,pee probably taste like salty tea someone dran...,0


In [111]:
df.drop(columns=['class'], inplace=True)
df

,cleaned_text,label
0,sex wife threatening suicide recently left wif...,1
1,weird not get affected compliment coming someo...,0
2,finally almost so never hear bad year ever swe...,0
3,need help help cry so hard,1
4,end tonight not anymore quit,1
...,...,...
183319,something today went sledding friend may not s...,0
183320,not like rock not going get anything but go,0
183321,tell so many friend not lonely everything depr...,0
183322,pee probably taste like salty tea someone dran...,0


In [112]:
train, test = train_test_split(df,
                               random_state=SEED,
                               test_size=0.2,
                               stratify=df['label'])

val, test = train_test_split(test,
                             random_state=SEED,
                             test_size=0.5,
                             stratify=test['label'])

In [113]:
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification

tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")
model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased")

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.weight', 'pre_classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [114]:
def dataset_conversion(train, test, val):

  train.reset_index(drop=True, inplace=True)
  test.reset_index(drop=True, inplace=True)
  val.reset_index(drop=True, inplace=True)

  train_dataset = Dataset.from_pandas(train)
  test_dataset = Dataset.from_pandas(test)
  val_dataset = Dataset.from_pandas(val)

  return DatasetDict({"train": train_dataset,
                      "test": test_dataset,
                      "val": val_dataset})


In [115]:
raw_datasets = dataset_conversion(train, test, val)

In [116]:
def tokenize_function(dataset):
    return tokenizer(dataset["cleaned_text"], padding="max_length", truncation=True)


In [117]:
tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)

Map:   0%|          | 0/146659 [00:00<?, ? examples/s]

Map:   0%|          | 0/18333 [00:00<?, ? examples/s]

Map:   0%|          | 0/18332 [00:00<?, ? examples/s]

In [118]:
# SAMPLE_SIZE = 20
# small_train_dataset = tokenized_datasets["train"].shuffle(seed=SEED).select(range(SAMPLE_SIZE))
# small_test_dataset = tokenized_datasets["test"].shuffle(seed=SEED).select(range(SAMPLE_SIZE))
# small_val_dataset = tokenized_datasets["val"].shuffle(seed=SEED).select(range(SAMPLE_SIZE))
train_dataset = tokenized_datasets["train"]
test_dataset = tokenized_datasets["test"]
val_dataset = tokenized_datasets["val"]

In [119]:
def compute_metrics(eval_pred):
    metric_acc = load_metric("accuracy")
    metric_rec = load_metric("recall")
    metric_pre = load_metric("precision")
    metric_f1 = load_metric("f1")

    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)

    accuracy = metric_acc.compute(predictions=predictions, references=labels)["accuracy"]
    recall = metric_rec.compute(predictions=predictions, references=labels)["recall"]
    precision = metric_pre.compute(predictions=predictions, references=labels)["precision"]
    f1 = metric_f1.compute(predictions=predictions, references=labels)["f1"]

    return {"accuracy": accuracy, "recall": recall, "precision": precision, "f1": f1}

In [120]:
training_args = TrainingArguments(
    output_dir=MODEL_CHECKPOINT_PATH,
    overwrite_output_dir = True,
    report_to = 'wandb',
    learning_rate=LEARNING_RATE,
    num_train_epochs=EPOCHS,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    seed=SEED,
    run_name=WANDB_RUN,
    save_strategy="steps",
    save_steps=1500
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [121]:
wandb.login()

<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [122]:
wandb.init(project=WANDB_PROJECT, entity=WANDB_ENTITY, name=WANDB_RUN)

wandb: Currently logged in as: nilankabiswas21. Use `wandb login --relogin` to force relogin


In [123]:
trainer.train()

Step,Training Loss
500,0.368300
1000,0.333100
1500,0.300200
2000,0.270900
2500,0.274300
3000,0.261400
3500,0.280300
4000,0.267400
4500,0.256900
5000,0.258600


Step,Training Loss
500,0.368300
1000,0.333100
1500,0.300200
2000,0.270900
2500,0.274300
3000,0.261400
3500,0.280300
4000,0.267400
4500,0.256900
5000,0.258600


TrainOutput(global_step=24444, training_loss=0.24370519831262114, metrics={'train_runtime': 7506.1236, 'train_samples_per_second': 19.539, 'train_steps_per_second': 3.257, 'total_flos': 1.9427536219490304e+16, 'train_loss': 0.24370519831262114, 'epoch': 1.0})

In [124]:
wandb.finish()

train/epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
train/global_step,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
train/learning_rate,████▇▇▇▇▇▇▆▆▆▆▆▅▅▅▅▅▄▄▄▄▄▄▃▃▃▃▃▂▂▂▂▂▂▁▁▁
train/loss,█▇▅▄▄▄▄▃▃▃▃▄▃▃▃▄▃▃▂▃▃▃▂▂▂▂▃▂▂▄▃▁▂▃▂▁▂▃▁▁
train/total_flos,▁
train/train_loss,▁
train/train_runtime,▁
train/train_samples_per_second,▁
train/train_steps_per_second,▁
train/epoch,1.0
train/global_step,24444


In [125]:
trainer.save_model(MODEL_SAVE_PATH)

In [ ]:
trainer.evaluate()

In [ ]:
trainer.predict(test_dataset).metrics

In [133]:
import torch
from transformers import AutoModelForSequenceClassification, AutoTokenizer

MODEL_SAVE_PATH = "/content/drive/MyDrive/Text_pred/Model/distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(MODEL_SAVE_PATH)
model = AutoModelForSequenceClassification.from_pretrained(MODEL_SAVE_PATH)

text = "this world is so cool I don't want to leave here I don't know what I think it but I want to die please help me I don't want to leave here"
# my_text = "hi I am so happy today I don't know what to do I don't want to leave here but still I am going to do something else funny"

inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True)

# Make the prediction
with torch.no_grad():
    outputs = model(**inputs)

predicted_probabilities = torch.sigmoid(outputs.logits)

predicted_probability_positive_class = predicted_probabilities[0][1].item()


predicted_label = torch.argmax(outputs.logits, dim=1).item()
predicted_class = ""
if predicted_label == 1 :
  predicted_class = "Suicidal"
else :
  predicted_class = "Not Suicidal"
# Print the result
print(f"Speaker is {predicted_class} with {predicted_probability_positive_class:.2f} probability")

Predicted Probability for the Positive Class: 0.8661402463912964
Speaker is Suicidal with 0.87 probability


In [134]:
predicted_probability_positive_class

0.8661402463912964

In [135]:
predicted_probabilities

tensor([[0.1302, 0.8661]])